In [3]:
# When working in AI LRZ
%cd ~/cma/CMA_Fairness_v2

/dss/dsshome1/0C/ra93lal2/cma/CMA_Fairness_v2


/dss/dsshome1/0C/ra93lal2/.local/share/virtualenvs/CMA_Fairness_v2-3j10GkSs/lib/python3.10/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
/dss/dsshome1/0C/ra93lal2/.local/share/virtualenvs/CMA_Fairness_v2-3j10GkSs/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [23]:
import pandas as pd

# Configuration

In [4]:
# Auto-reload the custom package
%load_ext autoreload
%autoreload 1
%aimport fairness_multiverse


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
RUN_TO_ANALYSE = "8"

In [12]:
PREFIX_SETTINGS = "sett_"
PREFIX_EVAL = "sett_eval_"
PREFIX_PERFORMANCE = "perf_"
PREFIX_FAIRNESS = "fair_"

In [13]:
from pathlib import Path

OUTPUT_DIR = Path(".") / "output"

# Directory that will contain outputs from analysis
ANALYSIS_OUTPUT_DIR = OUTPUT_DIR / "analyses" / (str(RUN_TO_ANALYSE))
ANALYSIS_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

DATA_DIR = OUTPUT_DIR / "runs" / str(RUN_TO_ANALYSE) / "data"

In [14]:
main_fairness_metric = "fair_main_equalized_odds_difference"

## Data Loading

In [15]:
df_agg_raw = pd.read_csv(DATA_DIR / f"agg_{RUN_TO_ANALYSE}_run_outputs.csv.gz")

In [16]:
import json

df_settings = pd.json_normalize(
    df_agg_raw["universe_settings"].apply(json.loads)
).add_prefix(
    PREFIX_SETTINGS
)

df_agg_full = df_settings.join(df_agg_raw)
df_agg_full.head()

,sett_cutoff,sett_eval_fairness_grouping,sett_exclude_features,sett_exclude_subgroups,sett_model,Unnamed: 0,run_no,universe_id,universe_settings,execution_time,...,perf_grp_precision_0,perf_grp_precision_1,perf_grp_false positive rate_0,perf_grp_false positive rate_1,perf_grp_false negative rate_0,perf_grp_false negative rate_1,perf_grp_selection rate_0,perf_grp_selection rate_1,perf_grp_count_0,perf_grp_count_1
0,quantile_0.1,majority-minority,sex,keep-all,gbm,0,8,d666e2b26ad834a3a31caefc780fa30b,"{""cutoff"": ""quantile_0.1"", ""eval_fairness_grou...",50.738868,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,quantile_0.1,nationality-all,sex,keep-all,gbm,1,8,d666e2b26ad834a3a31caefc780fa30b,"{""cutoff"": ""quantile_0.1"", ""eval_fairness_grou...",50.738868,...,0.119908,0.14770,0.949978,0.867833,0.006757,0.012254,0.954966,0.883678,20540.0,69170.0
2,quantile_0.25,majority-minority,sex,keep-all,gbm,2,8,d666e2b26ad834a3a31caefc780fa30b,"{""cutoff"": ""quantile_0.25"", ""eval_fairness_gro...",50.738868,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,quantile_0.25,nationality-all,sex,keep-all,gbm,3,8,d666e2b26ad834a3a31caefc780fa30b,"{""cutoff"": ""quantile_0.25"", ""eval_fairness_gro...",50.738868,...,0.134721,0.17183,0.789236,0.699550,0.057010,0.046718,0.806962,0.733078,20540.0,69170.0
4,quantile_0.1,majority-minority,sex,drop-non-german,logreg,4,8,aeda81ea848062c8b09ab80466cf07c3,"{""cutoff"": ""quantile_0.1"", ""eval_fairness_grou...",22.508544,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
rows, columns = df_agg_full.shape
print(f"The data has N = {rows} rows and N = {columns} columns.")

The data has N = 128 rows and N = 56 columns.


In [18]:
cols_settings = list(df_agg_full.columns[df_agg_full.columns.str.startswith(PREFIX_SETTINGS)])
cols_eval = list(df_agg_full.columns[df_agg_full.columns.str.startswith(PREFIX_EVAL)])
cols_non_eval = list(set(cols_settings) - set(cols_eval))
cols_performance = list(df_agg_full.columns[df_agg_full.columns.str.startswith(PREFIX_PERFORMANCE)])
cols_fairness = list(df_agg_full.columns[df_agg_full.columns.str.startswith(PREFIX_FAIRNESS)])


In [19]:
df_agg_full["universe_id"]

0      d666e2b26ad834a3a31caefc780fa30b
1      d666e2b26ad834a3a31caefc780fa30b
2      d666e2b26ad834a3a31caefc780fa30b
3      d666e2b26ad834a3a31caefc780fa30b
4      aeda81ea848062c8b09ab80466cf07c3
                     ...               
123    8b2c11c19583e4d5b9a213b53e76730c
124    37cb643ae0e5b9b35a6d97f8a210c57b
125    37cb643ae0e5b9b35a6d97f8a210c57b
126    37cb643ae0e5b9b35a6d97f8a210c57b
127    37cb643ae0e5b9b35a6d97f8a210c57b
Name: universe_id, Length: 128, dtype: object

In [20]:
drop_mask = df_agg_full["sett_exclude_subgroups"].isin(['drop-non-german'])
print(f"Dropping N = {drop_mask.sum()} rows, keeping N = {(~drop_mask).sum()}")

df_agg_full = df_agg_full[~drop_mask]

Dropping N = 64 rows, keeping N = 64


In [21]:
df_agg_full["sett_exclude_subgroups"].unique()

array(['keep-all'], dtype=object)

In [22]:
df_agg_full["sett_eval_fairness_grouping"].unique()

array(['majority-minority', 'nationality-all'], dtype=object)

In [ ]:
#df_agg_full["sett_eval_exclude_subgroups"].unique()

In [ ]:
#df_agg_full["sett_eval_on_subset"].unique()